In [0]:
import tensorflow as tf
import math
import numpy as np
import random
import librosa
import sklearn
import matplotlib.pyplot as plt
import librosa.display

In [0]:
test_1 = 'test_x_01.wav'
output_1 = 'output_1.wav'

test_2 = 'test_x_02.wav'
output_2 = 'output_2.wav'

save_path = "speech_denoising.ckpt/"

x = tf.placeholder("float", [None,513])
y = tf.placeholder("float", [None,513])

# prob = tf.placeholder_with_default(1.0, shape=())


In [3]:

x_input = tf.reshape(x, shape=[-1, 20, 513, 1])

# Convolution Layer with 32 filters and a kernel size of 5
conv1 = tf.layers.conv2d(inputs=x_input, filters=16, kernel_size=[4,4], activation = tf.nn.relu,  padding="same")
# Max Pooling (down-sampling) with strides of 2 and kernel size of 2
max1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=[2,2])

# Convolution Layer with 64 filters and a kernel size of 3
conv2 = tf.layers.conv2d(inputs=max1, filters=32, kernel_size=[2,2], activation = tf.nn.relu , padding="same")
# Max Pooling (down-sampling) with strides of 2 and kernel size of 2
max2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=[2,2])

# Flatten the data to a 1-D vector for the fully connected layer
fc1 = tf.layers.flatten(max2)

# Fully connected layer (in tf contrib folder for now)
#fc1 = tf.layers.dense(fc1, 2048)
# Apply Dropout (if is_training is False, dropout is not applied)
#fc1 = tf.layers.dropout(fc1, rate=prob)

# Output layer, class prediction
out = tf.layers.dense(fc1, 513,activation=tf.nn.relu)


Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


In [4]:
loss=tf.losses.mean_squared_error(labels=y,predictions=out)

step = tf.Variable(0, trainable=False)
rate = tf.train.exponential_decay(0.01, step, 1, 0.9999)

train = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(loss)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

Instructions for updating:
Use tf.cast instead.


In [5]:
s, sr = librosa.load('train_clean_male.wav', sr=None)
S = librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr = librosa.load('train_dirty_male.wav', sr=None)
X = librosa.stft(sn, n_fft=1024, hop_length=512)

S = np.abs(S)
S = S.T
S1 = S[19:,:]
X = np.abs(X)
X1 = X.T

temp = X1[0:20,:]

batch_size = 128
n_input = 2459

print(np.shape(temp))
for i in range(1,np.shape(X1)[0]-20+1):
  temp2 = X1[i:i+20,:]
  temp = np.vstack((temp,temp2))

print(np.shape(temp))
X2 = np.reshape(temp,(2440,20,513))

(20, 513)
(48800, 513)


In [6]:
print(S.shape)

(2459, 513)


In [7]:
print(np.shape(X))

(513, 2459)


In [0]:
def train_model():
    saver=tf.train.Saver()
      
    for i in range(2000):
      for j in range((int)(n_input/batch_size)):
        X_batch, S_batch = X2[j*batch_size:(j+1)*batch_size,:], S1[(j*batch_size):(j+1)*batch_size,:]
        X_batch = X_batch.reshape(np.shape(X_batch)[0]*np.shape(X_batch)[1],np.shape(X_batch)[2])
        
        train.run(feed_dict={x:X_batch, y:S_batch})  
      if i % 100 == 0:
          print("Epoch: ",i/100)
          print("Loss: ",loss.eval(feed_dict={x: X_batch, y:S_batch}))
          saver.save(sess, save_path)


In [9]:
train_model()

Epoch:  0.0
Loss:  0.024280395
Epoch:  1.0
Loss:  0.0022506858
Epoch:  2.0
Loss:  0.0019258334
Epoch:  3.0
Loss:  0.0017080025
Epoch:  4.0
Loss:  0.0013801641
Epoch:  5.0
Loss:  0.001334944
Epoch:  6.0
Loss:  0.0011429992
Epoch:  7.0
Loss:  0.0010448571
Epoch:  8.0
Loss:  0.00092139706
Epoch:  9.0
Loss:  0.000944798
Epoch:  10.0
Loss:  0.0007871535
Epoch:  11.0
Loss:  0.0007285615
Epoch:  12.0
Loss:  0.000821655
Epoch:  13.0
Loss:  0.0007877487
Epoch:  14.0
Loss:  0.0006439162
Epoch:  15.0
Loss:  0.00064165925
Epoch:  16.0
Loss:  0.0007163342
Epoch:  17.0
Loss:  0.0006283875
Epoch:  18.0
Loss:  0.0006101368
Epoch:  19.0
Loss:  0.0006489282


In [10]:
sn_t, sr_t = librosa.load(test_1, sr=None)
X_test = librosa.stft(sn_t, n_fft=1024, hop_length=512)

X_test2 = np.abs(X_test)

print(np.shape(X_test2))

temp_test = X_test2.T[0:20,:]

print(np.shape(temp_test))
for i in range(1,np.shape(X_test2.T)[0]-20+1):
  temp2 = X_test2.T[i:i+20,:]
  temp_test = np.vstack((temp_test,temp2))

print(np.shape(temp_test))


(513, 142)
(20, 513)
(2460, 513)


In [0]:
def test_model(test_file, output_file):
  sn_t, sr_t = librosa.load(test_file, sr=None)
  X_test = librosa.stft(sn_t, n_fft=1024, hop_length=512)
  X_test2 = np.abs(X_test)

#   print(np.shape(X_test2))

  temp_test1 = X_test2.T[0:20,:]

#   print(np.shape(temp_test1))
  for i in range(1,np.shape(X_test2.T)[0]-20+1):
    temp2 = X_test2.T[i:i+20,:]
    temp_test1 = np.vstack((temp_test1,temp2))

#   print(np.shape(temp_test))

#   print(X_test.shape)
  with sess.as_default():
    output = out.eval(feed_dict = {x: temp_test1})
  output_t = np.transpose(output)
#   print(output)
#   print(np.shape(output))
#   print(np.shape(output_t))
  silent1 = np.empty([19,513])
  silent1.fill(1e-12)
  temp1_silent1 = np.vstack((silent1,output))
#   print(temp1_silent1.shape)
  final_answer = np.multiply(np.divide(X_test, np.abs(X_test)).T, temp1_silent1)
#   print(final_answer.shape)

  final_answer = librosa.istft(final_answer.T, win_length=1024, hop_length=512)
  librosa.output.write_wav(output_file, final_answer, sr_t)

test_model(test_1,output_1)
test_model(test_2,output_2)

In [12]:
import IPython.display as ipd
ipd.Audio('./test_x_01.wav')

In [13]:
ipd.Audio('./output_1.wav')

In [14]:
ipd.Audio('./test_x_02.wav')

In [15]:
ipd.Audio('./output_2.wav')

In [16]:
x_train1,sr = librosa.load('train_dirty_male.wav',sr=None)
# print(np.shape(x_train1))
X_train1 = librosa.stft(x_train1, n_fft=1024, hop_length=512)
# print(np.shape(s))
        
x_train1_abs = np.abs(X_train1)
# print(np.shape(x_train1_abs))

temp_test3 = x_train1_abs.T[0:20,:]

# print(np.shape(temp_test3))
for i in range(1,np.shape(x_train1_abs.T)[0]-20+1):
  temp2 = x_train1_abs.T[i:i+20,:]
  temp_test3 = np.vstack((temp_test3,temp2))


with sess.as_default():
  predictions = out.eval(feed_dict = {x:temp_test3})
temp2=np.empty([19,513])

temp2.fill(1e-15)
predictions=np.vstack((temp2,predictions))
fact = X_train1.T/x_train1_abs.T
# print(np.shape(S))
# print(np.shape(x_train1_abs))

# print(np.shape(fact))
s_train1 = np.multiply(fact,predictions)
# print(np.shape(s_train1))
# print(s_train1)

sh_train1  = librosa.istft(s_train1.T , win_length = 1024 , hop_length=512)
# print(np.shape(sh_train1))

sh_train = s[0:1258496]
# print(np.shape(sh_train))

num = np.sum(sh_train ** 2)
den = np.sum((sh_train - sh_train1) ** 2)
SNR = 10 * np.log10(num/den)
print("SNR = ", SNR)
#sess.close()

SNR =  15.586470365524292
